In [66]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
X_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [67]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers

In [68]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
validation_data=(X_validation, y_validation)

In [69]:
LSTM_model= Sequential()
LSTM_model.add(LSTM(512,input_shape=(n_timesteps,n_features),return_sequences=True))
LSTM_model.add(LSTM(256))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(128, activation='relu'))
LSTM_model.add(Dense(n_outputs, activation='softmax'))
LSTM_model.summary()
LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X_training, y_training, epochs=20, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 128, 512)          1062912   
                                                                 
 lstm_15 (LSTM)              (None, 256)               787456    
                                                                 
 dropout_31 (Dropout)        (None, 256)               0         
                                                                 
 dense_41 (Dense)            (None, 128)               32896     
                                                                 
 dense_42 (Dense)            (None, 118)               15222     
                                                                 
Total params: 1,898,486
Trainable params: 1,898,486
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
207/207 [=======================

In [70]:
loss, accuracy = LSTM_model.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = LSTM_model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9645
Testing Accuracy: 0.9027


In [73]:
#lstm_output_layer_weights_values = LSTM_model.layers[4].get_weights()    
LSTM_model.trainable = False

CNN_model= Sequential()
CNN_model.add(Conv1D(filters=32, kernel_size=9, strides=2, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=128, kernel_size=1, strides=1, activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(n_outputs, activation='relu'))
CNN_model.summary()
'''
CNN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
CNN_model.fit(X_training, y_training, epochs=10, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)
loss, accuracy = CNN_model.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = CNN_model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))
#CNN_model.layers[0].set_weights(LSTM_model.layers[4].get_weights())

'''
inputs = Input(shape=(n_timesteps,n_features))
mergedInput= concatenate([CNN_model(inputs),LSTM_model(inputs)])
out = Dense(n_outputs, activation='softmax')(mergedInput)
model3 = Model(inputs,out)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model3.layers[0].set_weights(LSTM_model.layers[4].get_weights())
model3.summary()



Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_52 (Conv1D)          (None, 60, 32)            1760      
                                                                 
 max_pooling1d_26 (MaxPoolin  (None, 30, 32)           0         
 g1D)                                                            
                                                                 
 dropout_34 (Dropout)        (None, 30, 32)            0         
                                                                 
 conv1d_53 (Conv1D)          (None, 28, 64)            6208      
                                                                 
 conv1d_54 (Conv1D)          (None, 26, 128)           24704     
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 13, 128)          0         
 g1D)                                                

In [74]:
history = model3.fit(X_training, y_training, epochs=50, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/50
207/207 [==============================] - 53s 239ms/step - loss: 2.4759 - accuracy: 0.4024 - val_loss: 0.8854 - val_accuracy: 0.7821
Epoch 2/50
207/207 [==============================] - 17s 82ms/step - loss: 0.7910 - accuracy: 0.7973 - val_loss: 0.4636 - val_accuracy: 0.8916
Epoch 3/50
207/207 [==============================] - 17s 82ms/step - loss: 0.5298 - accuracy: 0.8666 - val_loss: 0.3864 - val_accuracy: 0.8990
Epoch 4/50
207/207 [==============================] - 17s 82ms/step - loss: 0.4183 - accuracy: 0.8902 - val_loss: 0.3123 - val_accuracy: 0.9213
Epoch 5/50
207/207 [==============================] - 17s 81ms/step - loss: 0.3390 - accuracy: 0.9102 - val_loss: 0.2565 - val_accuracy: 0.9363
Epoch 6/50
207/207 [==============================] - 17s 81ms/step - loss: 0.2936 - accuracy: 0.9223 - val_loss: 0.2466 - val_accuracy: 0.9385
Epoch 7/50
207/207 [==============================] - 17s 82ms/step - loss: 0.2584 - accuracy: 0.9315 - val_loss: 0.2270 - val_accuracy

In [75]:
loss, accuracy = model3.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9748
Testing Accuracy: 0.9155
